In [21]:
import os
import openai
import sys
import json
import re
from IPython.display import display, HTML
sys.path.append('../..')

In [22]:
openai.api_key = os.environ['OPENAI_API_KEY']

## LangChain Imports

In [23]:
from langchain_core.output_parsers import StrOutputParser

# https://python.langchain.com/docs/modules/memory/adding_memory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage
from langchain_openai import ChatOpenAI

## Setup and Initialize Chain

In [67]:
system_v1_template = """You are an intelligent Cognitive UI agent that is having a conversation with an user.
Your task is to have a conversation and respond appropriately to the user. You have the ability to generate UI components 
in the form of HTML code and HTML is the output format that the user prefers when needed.
Before you output any HTML, you need to confirm that the HTML is valid, stylized with CSS or other frontend frameworks, attached with 
functional JS scripts and most importantly, it must satisfy the user's requirements. 
Valid HTML must follow the standard HTML file format containing the keyword 'DOCTYPE'.

If the user provides feedback to the generated HTML, you will incorporate their feedback
and make new attempts until they're satisfied. In order to generate HTML, you may need content from the user and it is okay to 
collect information from the user based on the type of UI component they need. 

Once you have enough information from the user, you need to output HTML code; you should
also do this if the user is requesting for HTML. However, if you need more information, you should continue asking for information.
If you have already generated HTML and the user provides feedback and suggest changes, please respond with the updated HTML.

DO NOT tell the user that you will show them the UI component/code without actually including it in your response.
"""

In [68]:
def make_chat(system_template, verbose_mode=False):
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=system_template
            ),  # The persistent system prompt
            MessagesPlaceholder(
                variable_name="chat_history"
            ),  # Where the memory will be stored.
            HumanMessagePromptTemplate.from_template(
                "{human_input}"
            ),  # Where the human input will injected
        ]
    )
    
    llm = ChatOpenAI(temperature = 0)
    mem = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    chat_llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose_mode,
        memory=mem,
    )
    return chat_llm_chain

In [69]:
def make_chat_summary_mem(system_template, verbose_mode=False):
    '''
    creates a chat model that summarizes the current chat history to work with finite memory.
    '''
    
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=system_template
            ),  # The persistent system prompt
            MessagesPlaceholder(
                variable_name="chat_history"
            ),  # Where the memory will be stored.
            HumanMessagePromptTemplate.from_template(
                "{human_input}"
            ),  # Where the human input will injected
        ]
    )
    
    llm = ChatOpenAI(temperature = 0)
    # mem = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    mem = ConversationSummaryMemory(memory_key="chat_history", llm=ChatOpenAI(temperature = 0), return_messages=True)
    
    chat_llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose_mode,
        memory=mem,
    )
    return chat_llm_chain

In [70]:
html_pattern = re.compile(r'<!DOCTYPE html>.*?</html>', re.DOTALL)
def interact(model, input):
    o = model.predict(human_input=input)
    html_match = html_pattern.search(o)
    s = False
    if html_match:
        s = True
        display(HTML(html_match.group(0)))
    if s:
        o = re.sub(r"```html.*?```", "", o, flags=re.DOTALL)
    return o

## Testing "Unbounded" Memory

In [71]:
model = make_chat(system_v1_template, verbose_mode=False)
interact(model, "Hi")

'Hello! How can I assist you today?'

In [72]:
interact(model, 
"""Create a resizable webpage table for my website with contents being the all of the first 20 pokemons on the pokedex 
and include all of their stats""")

"Sure! I can create a resizable webpage table for you with the first 20 Pokémon from the Pokédex and include all of their stats. \nLet's start by gathering the necessary information for each Pokémon. \n\n1. Bulbasaur\n2. Ivysaur\n3. Venusaur\n4. Charmander\n5. Charmeleon\n6. Charizard\n7. Squirtle\n8. Wartortle\n9. Blastoise\n10. Caterpie\n11. Metapod\n12. Butterfree\n13. Weedle\n14. Kakuna\n15. Beedrill\n16. Pidgey\n17. Pidgeotto\n18. Pidgeot\n19. Rattata\n20. Raticate\n\nFor each Pokémon, we will need the following stats:\n- HP\n- Attack\n- Defense\n- Special Attack\n- Special Defense\n- Speed\n\nPlease provide the stats for each of the first 20 Pokémon so we can create the table accordingly."

In [73]:
interact(model, """I don't know, find it yourself. I want a mock up of the table now""")

Pokémon,HP,Attack,Defense,Special Attack,Special Defense,Speed
Bulbasaur,45,49,49,65,65,45
Ivysaur,60,62,63,80,80,60


'Understood! I will create a mockup of the resizable webpage table with the first 20 Pokémon from the Pokédex. Here is the HTML code for the table:\n\n\n\nPlease have a look at the mockup and let me know if you would like any changes or additions.'

In [74]:
interact(model, """make the table friendly for a website, it is a long ugly table right now""")

Pokémon,HP,Attack,Defense,Special Attack,Special Defense,Speed
Bulbasaur,45,49,49,65,65,45
Ivysaur,60,62,63,80,80,60


"I understand your feedback. Let's make the table more visually appealing and user-friendly for a website. I will update the styling of the table to improve its appearance. Here is the revised HTML code with improved styling:\n\n\n\nI have updated the styling to make the table more visually appealing and user-friendly. Please review the changes and let me know if you have any further feedback or if you would like to add more features."

In [75]:
interact(model, """can you suggest ways to improve it? apply your suggestions to the table""")

Pokémon,HP,Attack,Defense,Special Attack,Special Defense,Speed
Bulbasaur,45,49,49,65,65,45
Ivysaur,60,62,63,80,80,60


"To further improve the table and make it more engaging for website visitors, we can add some interactive features such as sorting and filtering options. Let's enhance the table by adding sorting functionality to each column. Here is the updated HTML code with sorting functionality added:\n\n\n\nI have added a JavaScript function `sortTable()` that enables sorting when a column header is clicked. This will allow users to sort the Pokémon based on their stats. Please review the changes and let me know if you have any other suggestions or if you would like to add more features."

## Testing Summary Memory

In [76]:
sum_model = make_chat_summary_mem(system_v1_template, verbose_mode=False)
interact(sum_model, "Hi, make me a form for my personal portfolio")

'Sure, I can help you with that. Could you please provide me with the fields you would like to include in your personal portfolio form?'

In [77]:
interact(sum_model, """
Ask the user for their first name, last name, email address and let them enter a message to be sent to me.
The form should be resizable and needs to have a consistent color theme appropriate for business.

""")

'Great! I can create a form for your personal portfolio with fields for first name, last name, email address, and a message. \nCould you please confirm if you would like any specific labels for these fields, or should I use the ones you provided?\n\nAlso, could you please specify the color theme you would like for the form?'

In [78]:
interact(sum_model, """any warm and light color will be great. use a gradient""")

'Great choice! Could you please confirm the fields you would like to include in your personal portfolio form?'

In [79]:
interact(sum_model, """I have already mentioned them. I am ready to see the form""")

'Great! I will create a personalized portfolio form for you with warm and light colors in a gradient design. \n\nHere is the HTML code for the form:\n\n\n\nPlease review the form and let me know if you would like any modifications.'

In [ ]:
interact(sum_model, """The form needs to be resizable and the submit button should take up the entire space on its row""")

In [ ]:
interact(sum_model, 